In [1]:
import pandas as pd

In [2]:
human = pd.read_csv("data/human.csv")
human['autom'] = ~ human['human']

human

,act_upd_by,human,autom
0,0,False,True
1,1,True,False
2,2,True,False
3,3,True,False
4,4,False,True
...,...,...,...
1307,1307,True,False
1308,1308,True,False
1309,1309,True,False
1310,1310,True,False


In [19]:
log = pd.read_csv("data/combined_event_log_anonymous.csv")

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_52215/4135780487.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  log = pd.read_csv("data/combined_event_log_anonymous.csv")


# Sanity checks

In [ ]:
log[~ log['act_upd_by'].isin(human['act_upd_by'])]

In [ ]:
human[~ human['act_upd_by'].isin(log['act_upd_by'])]

# Analysis

## Human / automated activities

In [20]:
log2 = log.merge(human, validate='many_to_one')
log2

,app_num,act_upd_date,act_upd_by,activity_id,group,activity,activity_status,cor,citz,activity_full_value,human,autom
0,0,2024-06-03 09:20:37.327,0,1-SQ1RI5F,NaN,Application Status,Prospective,Vietnam,Vietnam,Application Status - Prospective,False,True
1,0,2024-06-03 09:20:37.327,1,0,NaN,Misrepresentation,NIL,Vietnam,Vietnam,Misrepresentation - NIL,True,False
2,0,2024-06-03 09:20:37.327,1,0,NaN,Eligibility Assessment,NIL,Vietnam,Vietnam,Eligibility Assessment - NIL,True,False
3,0,2024-06-03 09:20:37.327,1,0,NaN,Final Assessment,NIL,Vietnam,Vietnam,Final Assessment - NIL,True,False
4,0,2024-06-03 09:20:37.327,1,0,NaN,Received,Date,Vietnam,Vietnam,Received - Date,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
543298,7733,2024-06-17 07:46:39.433,281,1-STC2A6X,NaN,Other Reqs Assessment,NIL,Belgium,Cameroon,Other Reqs Assessment - NIL,True,False
543299,7733,2024-06-26 08:11:25.043,281,1-STC2A6X,NaN,Eligibility Assessment,Review Required,Belgium,Cameroon,Eligibility Assessment - Review Required,True,False
543300,7733,2024-06-28 10:45:09.050,281,1-STC2A6X,NaN,Eligibility Assessment,Failed,Belgium,Cameroon,Eligibility Assessment - Failed,True,False
543301,7733,2024-06-28 10:45:09.587,281,1-STC2A6X,NaN,Final Assessment,Refused,Belgium,Cameroon,Final Assessment - Refused,True,False


In [23]:
all_activ = log2['activity_full_value'].unique()
autom_activ = pd.Series(log2[log2['autom']==True]['activity_full_value'].unique())
human_activ = pd.Series(log2[log2['human']==True]['activity_full_value'].unique())

file = "data/Human vs automated activities.csv"

# only autom
autom_activ[~ autom_activ.isin(human_activ)].to_csv(file, index=False)

# only human
human_activ[~ human_activ.isin(autom_activ)].to_csv(file, mode='a', index=False)

# both autom & human
human_activ[human_activ.isin(autom_activ)].to_csv(file, mode='a', index=False)

In [6]:
actors_activs = log.groupby('act_upd_by')['activity'].apply(lambda g: tuple(g.drop_duplicates()))
actors_activs.to_excel("data/Actors and activities.xlsx")

## Full process: human / automated

In [ ]:
log = pd.read_csv("data/combined_event_log_anonymous.csv")

In [ ]:
log2 = log.merge(human, validate='many_to_one')

In [ ]:
case_human = log2.groupby('app_num')['human'].any()
case_human[case_human==True]

In [ ]:
case_autom = log2.groupby('app_num')['autom'].all()
case_autom[case_autom==True]

In [ ]:
case_human = log2.groupby('app_num')['human'].all()
case_human[case_human==True]

## Subprocesses: human / automated

In [7]:
import os
from pathlib import Path

deg_autom = []

dir = "/Users/wvw/git/pm/ircc/lifecycles"
for subdir in [ "level1", "level2" ]:
    for path in Path(os.path.join(dir, subdir, "logs")).rglob("*.csv"):
        log = pd.read_csv(path)
        name = os.path.basename(path)[:-4]
        log = log.merge(human, validate='many_to_one')
        case_cnt = log['case:concept:name'].nunique()
        autom_cases = log.groupby('case:concept:name')['autom'].all()
        
        only_autom_cases = autom_cases[autom_cases]
        num_autom = round(only_autom_cases.shape[0] / case_cnt, 2)
        deg_autom.append([ name, num_autom ])
        # print(name)
        # print("# autom:", num_autom)

deg_autom = pd.DataFrame(deg_autom, columns=[ 'name', 'deg_autom' ]).sort_values(by='deg_autom', ascending=False)

In [9]:
deg_autom.to_excel("data/Subprocess automation.xlsx", index=False)

In [18]:
# (quick sanity check)

log = pd.read_csv("/Users/wvw/git/pm/ircc/lifecycles/level2/logs/Medical.csv")
log = log.merge(human, validate='many_to_one')

num_cases = log['case:concept:name'].nunique()
num_non_autom = log[log['human']]['case:concept:name'].nunique()
(num_cases - num_non_autom) / num_cases

0.1761998685075608